# Basic Neural Network Model


## Artificial neuron

Recall the concept of a [neuron](https://en.wikipedia.org/wiki/Artificial_neuron) based on its mathematical formula.

$$ y_k = \varphi \left( \sum_{j=0}^{m}{w_{kj}x_j} +b_k \right) $$

This is a simple **linear** neuron. If you look closely, you will see the formula for multiple linear regression (if $\varphi$ is removed)! If $\varphi$ is a sigmoid funciton then it becomes the formula for losistic regression. 

PyTorch, as well as other NN packages, support numerous types of neurons. Typically, neurons are composed into layers, and a single layer has only a single type of neuron.

In this lab, we devlop linear regression and logistic regression models with neural networks.


## Tools 

**PyTorch**: In this lab, we are going to drill down into some neural network basics using the PyTorch package.

**What is PyTorch?**

From PyTorch official [site](https://pytorch.org/): 

> PyTorch is an open source machine learning library based on the Torch library, used for applications such as computer vision and natural language processing, primarily developed by Facebook's AI Research lab. It is free and open-source software released under the Modified BSD license

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt

import os, sys
import itertools
import numpy as np
import pandas as pd

from sklearn.preprocessing import scale, LabelBinarizer, StandardScaler, Normalizer
from sklearn.metrics import f1_score, confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn import datasets

# Random seed for numpy
np.random.seed(18937)

## Developing a Multiple Regression Model using Neural Network

Let's explore the Boston housing dataset which is used in a regression setting. 

In [2]:
dataset = datasets.load_boston()
dataset.keys()

dict_keys(['data', 'target', 'feature_names', 'DESCR', 'filename'])

In [3]:
df = pd.DataFrame(dataset.data, columns=dataset.feature_names)
df['Price'] = dataset.target
df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,Price
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


In [4]:
df.describe()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,Price
count,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000
mean,3.613524,11.363636,11.136779,0.069170,0.554695,6.284634,68.574901,3.795043,9.549407,408.237154,18.455534,356.674032,12.653063,22.532806
std,8.601545,23.322453,6.860353,0.253994,0.115878,0.702617,28.148861,2.105710,8.707259,168.537116,2.164946,91.294864,7.141062,9.197104
min,0.006320,0.000000,0.460000,0.000000,0.385000,3.561000,2.900000,1.129600,1.000000,187.000000,12.600000,0.320000,1.730000,5.000000
25%,0.082045,0.000000,5.190000,0.000000,0.449000,5.885500,45.025000,2.100175,4.000000,279.000000,17.400000,375.377500,6.950000,17.025000
50%,0.256510,0.000000,9.690000,0.000000,0.538000,6.208500,77.500000,3.207450,5.000000,330.000000,19.050000,391.440000,11.360000,21.200000
75%,3.677083,12.500000,18.100000,0.000000,0.624000,6.623500,94.075000,5.188425,24.000000,666.000000,20.200000,396.225000,16.955000,25.000000
max,88.976200,100.000000,27.740000,1.000000,0.871000,8.780000,100.000000,12.126500,24.000000,711.000000,22.000000,396.900000,37.970000,50.000000


## Standardization/Normalization of Data

In [5]:
scaler = Normalizer()
data_scaled = scaler.fit_transform(df)
df_scaled = pd.DataFrame(data_scaled, columns=list(dataset.feature_names) + ['Price'])
df_scaled.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,Price
0,0.000013,0.035955,0.004614,0.0,0.001075,0.013134,0.130238,0.008170,0.001998,0.591265,0.030562,0.792814,0.009948,0.047940
1,0.000058,0.000000,0.014961,0.0,0.000992,0.013588,0.166966,0.010511,0.004232,0.512112,0.037668,0.839907,0.019342,0.045709
2,0.000058,0.000000,0.015133,0.0,0.001004,0.015379,0.130778,0.010632,0.004281,0.517974,0.038099,0.840809,0.008626,0.074272
3,0.000071,0.000000,0.004772,0.0,0.001003,0.015319,0.100257,0.013270,0.006567,0.485964,0.040935,0.863856,0.006436,0.073114
4,0.000150,0.000000,0.004740,0.0,0.000996,0.015539,0.117842,0.013181,0.006523,0.482675,0.040658,0.862945,0.011589,0.078707


## Split training data and testing data

In [6]:
X = df_scaled.drop('Price', axis=1).to_numpy()
y = df_scaled['Price'].to_numpy()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=23)

## Construct a neural network

Now we will construct a basic Neural Network with
 * One hidden layer fed by 13 input values (as there are 13 features)
 * One output layer 
 
##### Note: The summary will show that we have 16 total learnable parameters:
  * 14 for the hidden layer (13 feature values and bias)
  * 1 for the output layer (Hidden ($H_0$) and without bias) 
  

<figure>
  <img src="../images/reg_as_nn.jpg" width=600 height=400 alt="figure alt text">
  <figcaption>
      <b>Fig. A neural network for solving muliple regression problem.</b> <!-- can also use <div>, <p>, etc. tags within <figcaption> -->
  </figcaption>
</figure>

In [7]:
# load necessary pytorch modules
import torch
from torch import nn
from torch import optim
from torch.utils.data import TensorDataset, DataLoader

### Defining the Model

One way to define a neural network in PyTorch is to subclass the `nn.Module` class. 


In [8]:
class MyRegNN(nn.Module):
    def __init__(self, D_in, H, D_out):
        """
        D_in: number of input
        """
        super(MyRegNN, self).__init__()
        self.layer1 = nn.Linear(D_in, H) # input to hidden layer
        self.layer2 = nn.Linear(H, D_out, bias=False) # input to hidden layer
        
    def forward(self, x):
        h_pred = self.layer1(x)        # h = dot(input,w1) 
        y_pred = self.layer2(h_pred)   
        return y_pred


Now, we create an instance of the network class we have created. 

In [9]:
# here is a network with 13 inputs to 1 hidden neurons to one output neuron 

D_in, H, D_out = X_train.shape[1], 1, 1    

net = MyRegNN(D_in, H, D_out)

We can summarize this model using `summary` function from `torchsummary` package. 

In [10]:
!pip install torchsummary

In [11]:
from torchsummary import summary

In [12]:
summary(net, (X_train.shape[1],))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                    [-1, 1]              14
            Linear-2                    [-1, 1]               1
Total params: 15
Trainable params: 15
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00
----------------------------------------------------------------


### Define Loss Function and Optimizer

In [13]:
loss = nn.MSELoss()   # Mean Squared error loss
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.3)  
# optimizer = optim.SGD(net.parameters(), lr=0.001)  

### Training the Model

Before training the model, we need to convert the pandas/numpy datasets to pytorch's tensor data structure.

In [14]:
# create tensors from the train/test set
X_train_tensor = torch.tensor(X_train, dtype=torch.float)
X_test_tensor = torch.tensor(X_test, dtype=torch.float)
y_train_tensor = torch.tensor(y_train, dtype=torch.float).view(-1, 1)
y_test_tensor = torch.tensor(y_test, dtype=torch.float).view(-1, 1)

For better iteration over the train/test sets, there are two handy methods: TensorDataset and DataLoader.

In [15]:
BATCH_SIZE = 1  # it is possible to feed more than one istances to the model. 
# These set of instances is called batch. For simplicity, let's keep one instance per batch

train_data = TensorDataset(X_train_tensor, y_train_tensor)
test_data = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(dataset=test_data, batch_size=1)

Now, we train the mdoel with 100 epochs. The number of epochs is a hyperparameter that defines the number times that the learning algorithm will work through the entire training dataset. Within an epoch each sample in the training dataset has had an opportunity to update the internal model parameters. An epoch is comprised of one or more batches. 

Note: For simplicity we are skipping k-fold cross validation. 

In [16]:
N_EPOCHS = 100  # In each epoch, the model iterate over all the instances 

for epoch in range(N_EPOCHS):
    epoch_loss = 0
    
    for x, y in train_loader:
        output = net(x)        # Forward pass: get the network output for this instance
        l = loss(output, y)    # estimate error for this instance
        epoch_loss += l.item() # Aggregate error
        optimizer.zero_grad()  # As backward method accumulates gradient, we need to set it to 0
        l.backward()           # Backward pass: Estimate gradient 
        optimizer.step()

    if (epoch%5)==0:
        print(f'Epoch {epoch+0:03}: | Total Loss: {epoch_loss:.5f} | Avg Loss: {epoch_loss/len(train_loader):.5f}')

Epoch 000: | Total Loss: 0.80323 | Avg Loss: 0.00187
Epoch 005: | Total Loss: 0.51578 | Avg Loss: 0.00120
Epoch 010: | Total Loss: 0.39139 | Avg Loss: 0.00091
Epoch 015: | Total Loss: 0.30770 | Avg Loss: 0.00072
Epoch 020: | Total Loss: 0.24940 | Avg Loss: 0.00058
Epoch 025: | Total Loss: 0.20783 | Avg Loss: 0.00048
Epoch 030: | Total Loss: 0.17789 | Avg Loss: 0.00041
Epoch 035: | Total Loss: 0.15574 | Avg Loss: 0.00036
Epoch 040: | Total Loss: 0.13926 | Avg Loss: 0.00032
Epoch 045: | Total Loss: 0.12686 | Avg Loss: 0.00030
Epoch 050: | Total Loss: 0.11739 | Avg Loss: 0.00027
Epoch 055: | Total Loss: 0.11027 | Avg Loss: 0.00026
Epoch 060: | Total Loss: 0.10472 | Avg Loss: 0.00024
Epoch 065: | Total Loss: 0.10046 | Avg Loss: 0.00023
Epoch 070: | Total Loss: 0.09714 | Avg Loss: 0.00023
Epoch 075: | Total Loss: 0.09456 | Avg Loss: 0.00022
Epoch 080: | Total Loss: 0.09253 | Avg Loss: 0.00022
Epoch 085: | Total Loss: 0.09095 | Avg Loss: 0.00021
Epoch 090: | Total Loss: 0.08966 | Avg Loss: 0

# Prediction with the model

In [17]:
net.eval()  # notify all the layers that we are in eval mode

with torch.no_grad(): 
    y_test_pred = net(X_test_tensor)


In [18]:
from sklearn.metrics import r2_score, mean_squared_error

print(f"R^2: {r2_score(y_test, y_test_pred.numpy())}")
print(f"MSE:{mean_squared_error(y_test, y_test_pred.numpy())}" )

R^2: 0.43304415737722446
MSE:0.0002304868692226775


In terms of MSE and R^2, the neural network performed better than the baseline which predicts mean as an output.

## Developing a Logistic Regression Model using Neural Network

For this lab, we will use sklearn breast cancer dataset. 

In [19]:
cancer = datasets.load_breast_cancer()
cancer.keys()

dict_keys(['data', 'target', 'target_names', 'DESCR', 'feature_names', 'filename'])

In [20]:
df = pd.DataFrame(cancer.data, columns=cancer.feature_names)
df['class'] = cancer.target
df.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,class
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,0


## Standardization/Normalization of Data

In [21]:
X = df.drop('class', axis=1).to_numpy()
y = df['class'].to_numpy()

scaler = Normalizer()
X_scaled = scaler.fit_transform(X)
df_scaled = pd.DataFrame(X_scaled, columns=list(cancer.feature_names))
df_scaled['class'] = y
df_scaled.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,class
0,0.007925,0.004573,0.054099,0.440986,0.000052,0.000122,0.000132,0.000065,0.000107,0.000035,...,0.007635,0.081325,0.889462,0.000071,0.000293,0.000314,0.000117,0.000203,0.000052,0
1,0.008666,0.007486,0.055988,0.558619,0.000036,0.000033,0.000037,0.000030,0.000076,0.000024,...,0.009862,0.066899,0.824026,0.000052,0.000079,0.000102,0.000078,0.000116,0.000038,0
2,0.009367,0.010109,0.061842,0.572276,0.000052,0.000076,0.000094,0.000061,0.000098,0.000029,...,0.012145,0.072545,0.812984,0.000069,0.000202,0.000214,0.000116,0.000172,0.000042,0
3,0.016325,0.029133,0.110899,0.551922,0.000204,0.000406,0.000345,0.000150,0.000371,0.000139,...,0.037881,0.141333,0.811515,0.000300,0.001238,0.000982,0.000368,0.000949,0.000247,0
4,0.009883,0.006985,0.065808,0.631774,0.000049,0.000065,0.000096,0.000051,0.000088,0.000029,...,0.008120,0.074137,0.767189,0.000067,0.000100,0.000195,0.000079,0.000115,0.000037,0


In [22]:
# class distribution
df_scaled['class'].value_counts()

1    357
0    212
Name: class, dtype: int64

## Split training data and testing data

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.15, random_state=23)

In [24]:
X.shape

(569, 30)

## Construct a neural network

Now we will construct a basic Neural Network with
 * One hidden layer fed by 30 input values (as there are 30 features)
 * One output layer 


In [25]:
class MyLogitNN(nn.Module):
    def __init__(self, D_in, H, D_out):
        """
        D_in: number of input
        """
        super(MyLogitNN, self).__init__()
        self.layer1 = nn.Linear(D_in, H) # input to hidden layer
        self.layer2 = nn.Linear(H, D_out, bias=False) # input to hidden layer
        
    def forward(self, x):
        h_pred = self.layer1(x)        
        y_pred = torch.sigmoid(self.layer2(h_pred))   
        return y_pred


Now, we create an instance of the network class we have created. 

In [26]:
# here is a network with 13 inputs to 1 hidden neurons to one output neuron 

D_in, H, D_out = X_train.shape[1], 1, 1    

net = MyLogitNN(D_in, H, D_out)

We can summarize this model using `summary` function from `torchsummary` package. 

In [27]:
summary(net, (X_train.shape[1],))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                    [-1, 1]              31
            Linear-2                    [-1, 1]               1
Total params: 32
Trainable params: 32
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00
----------------------------------------------------------------


### Define Loss Function and Optimizer

In [28]:
# loss = nn.MSELoss()   
loss = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(net.parameters(), lr=0.01)
# optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.3)  
# optimizer = optim.SGD(net.parameters(), lr=0.001)  



### Training the Model

Before training the model, we need to convert the pandas/numpy datasets to pytorch's tensor data structure.

In [29]:
# create tensors from the train/test set
X_train_tensor = torch.tensor(X_train, dtype=torch.float)
X_test_tensor = torch.tensor(X_test, dtype=torch.float)
y_train_tensor = torch.tensor(y_train, dtype=torch.float).view(-1, 1)
y_test_tensor = torch.tensor(y_test, dtype=torch.float).view(-1, 1)

For better iteration over the train/test sets, there are two handy methods: TensorDataset and DataLoader.

In [30]:
BATCH_SIZE = 1  # it is possible to feed more than one istances to the model. 
# These set of instances is called batch. For simplicity, let's keep one instance per batch

train_data = TensorDataset(X_train_tensor, y_train_tensor)
test_data = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(dataset=test_data, batch_size=1)

Now, we train the mdoel with 100 epochs. The number of epochs is a hyperparameter that defines the number times that the learning algorithm will work through the entire training dataset. Within an epoch each sample in the training dataset has had an opportunity to update the internal model parameters. An epoch is comprised of one or more batches. 

Note: For simplicity we are skipping k-fold cross validation. 

In [31]:
N_EPOCHS = 100  # In each epoch, the model iterate over all the instances 

for epoch in range(N_EPOCHS):
    epoch_loss = 0
    
    for x, y in train_loader:
        output = net(x)        # Forward pass: get the network output for this instance
        l = loss(output, y)    # estimate error for this instance
        epoch_loss += l.item() # Aggregate error
        optimizer.zero_grad()  # As backward method accumulates gradient, we need to set it to 0
        l.backward()           # Backward pass: Estimate gradient 
        optimizer.step()

    if (epoch%5)==0:
        print(f'Epoch {epoch+0:03}: | Total Loss: {epoch_loss:.5f} | Avg Loss: {epoch_loss/len(train_loader):.5f}')

Epoch 000: | Total Loss: 319.79717 | Avg Loss: 0.66211
Epoch 005: | Total Loss: 258.50843 | Avg Loss: 0.53521
Epoch 010: | Total Loss: 251.01117 | Avg Loss: 0.51969
Epoch 015: | Total Loss: 248.16098 | Avg Loss: 0.51379
Epoch 020: | Total Loss: 246.24395 | Avg Loss: 0.50982
Epoch 025: | Total Loss: 245.33038 | Avg Loss: 0.50793
Epoch 030: | Total Loss: 246.11768 | Avg Loss: 0.50956
Epoch 035: | Total Loss: 243.98878 | Avg Loss: 0.50515
Epoch 040: | Total Loss: 242.69297 | Avg Loss: 0.50247
Epoch 045: | Total Loss: 246.09537 | Avg Loss: 0.50951
Epoch 050: | Total Loss: 243.81669 | Avg Loss: 0.50480
Epoch 055: | Total Loss: 243.91049 | Avg Loss: 0.50499
Epoch 060: | Total Loss: 243.85692 | Avg Loss: 0.50488
Epoch 065: | Total Loss: 243.87989 | Avg Loss: 0.50493
Epoch 070: | Total Loss: 243.46901 | Avg Loss: 0.50408
Epoch 075: | Total Loss: 242.89966 | Avg Loss: 0.50290
Epoch 080: | Total Loss: 243.53383 | Avg Loss: 0.50421
Epoch 085: | Total Loss: 245.68934 | Avg Loss: 0.50867
Epoch 090:

# Prediction with the model

In [32]:
net.eval()  # notify all the layers that we are in eval mode

with torch.no_grad(): 
    y_test_pred = net(X_test_tensor)
    
y_test_pred[:5]

tensor([[9.9483e-01],
        [1.2563e-02],
        [3.5066e-11],
        [3.6112e-02],
        [1.1795e-13]])

In [33]:
y_test_pred_class = torch.round(y_test_pred)

In [34]:
from sklearn.metrics import classification_report, confusion_matrix

print(f"Confusion Matrix:\n {confusion_matrix(y_test, y_test_pred_class.numpy())}")
print(f"Classification Report:\n {classification_report(y_test, y_test_pred_class.numpy())}" )

Confusion Matrix:
 [[26  1]
 [14 45]]
Classification Report:
               precision    recall  f1-score   support

           0       0.65      0.96      0.78        27
           1       0.98      0.76      0.86        59

   micro avg       0.83      0.83      0.83        86
   macro avg       0.81      0.86      0.82        86
weighted avg       0.88      0.83      0.83        86



In this lab, we learned a step-by-step process for developing neural networks for solving regression and classification problems. These are elementary neural networks, but the process is similar even if our network architecture has more layers/neurons.  

---
# PyTorch API and helpful links

 * Layers: https://pytorch.org/docs/stable/nn.html
 * Loss / Loss Functions : [link1](https://medium.com/udacity-pytorch-challengers/a-brief-overview-of-loss-functions-in-pytorch-c0ddb78068f7) [link2](https://neptune.ai/blog/pytorch-loss-functions)
 * Optimizers (learning algorithm) : https://pytorch.org/docs/stable/optim.html
 * Neuron Activation Functions : https://towardsdatascience.com/understanding-pytorch-activation-functions-the-maths-and-algorithms-part-1-7d8ade494cee
 

### Please restart the kernel and clear all output, then play around with parameters or add cells and create additional notebooks

# Save your notebook